In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns


In [2]:

def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()




I use the Ticker function to extract the data to create a ticker object. The stock I'm choosing for now is Tesla, with its ticker symbol TSLA.

In [3]:
tesla = yf.Ticker("TSLA")

tesla_shares = tesla.history(period="max")

tesla_shares.reset_index(inplace=True)
tesla_shares.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
2955,2022-03-24,1009.729980,1024.489990,988.799988,1013.919983,22973600,0,0.0
2956,2022-03-25,1008.000000,1021.799988,997.320007,1010.640015,20642900,0,0.0
2957,2022-03-28,1065.099976,1097.880005,1053.599976,1091.839966,34168700,0,0.0
2958,2022-03-29,1107.989990,1114.770020,1073.109985,1099.569946,24538300,0,0.0
2959,2022-03-30,1091.170044,1113.949951,1084.000000,1093.989990,19913800,0,0.0


In [4]:
data = requests.get("https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue").text

In [7]:
soup = BeautifulSoup(data, "html5lib")
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in soup.find_all("tbody")[1].find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    print(date)
    rev = col[1].text
    print(rev)
    tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)

2021-12-31
$17,719
2021-09-30
$13,757
2021-06-30
$11,958
2021-03-31
$10,389
2020-12-31
$10,744
2020-09-30
$8,771
2020-06-30
$6,036
2020-03-31
$5,985
2019-12-31
$7,384
2019-09-30
$6,303
2019-06-30
$6,350
2019-03-31
$4,541
2018-12-31

C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method 


$7,226
2018-09-30
$6,824
2018-06-30
$4,002
2018-03-31
$3,409
2017-12-31
$3,288
2017-09-30
$2,985
2017-06-30
$2,790
2017-03-31
$2,696
2016-12-31
$2,285
2016-09-30
$2,298
2016-06-30
$1,270
2016-03-31
$1,147
2015-12-31
$1,214
2015-09-30

C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method 


$937
2015-06-30
$955
2015-03-31
$940
2014-12-31
$957
2014-09-30
$852
2014-06-30
$769
2014-03-31
$621
2013-12-31
$615
2013-09-30
$431
2013-06-30
$405
2013-03-31
$562
2012-12-31
$306
2012-09-30
$50
2012-06-30

C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method 


$27
2012-03-31
$30
2011-12-31
$39
2011-09-30
$58
2011-06-30
$58
2011-03-31
$49
2010-12-31
$36
2010-09-30
$31
2010-06-30
$28
2010-03-31
$21
2009-12-31

2009-09-30
$46
2009-06-30
$27


C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":rev}, ignore_index = True)
C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\2334425985.py:9: FutureWarning: The frame.append method 

In [8]:
tesla_revenue.head()

,Date,Revenue
0,2021-12-31,"$17,719"
1,2021-09-30,"$13,757"
2,2021-06-30,"$11,958"
3,2021-03-31,"$10,389"
4,2020-12-31,"$10,744"


In [10]:
#Removing the $ and , from the Revenue columns, and any other miscellanious items, like null values 
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue.head()

C:\Users\Rizky Gamal\AppData\Local\Temp\ipykernel_16872\485504536.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")


,Date,Revenue
0,2021-12-31,17719
1,2021-09-30,13757
2,2021-06-30,11958
3,2021-03-31,10389
4,2020-12-31,10744
